In [12]:
import pandas as pd
import seaborn as sb
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
X = np.load('/content/drive/MyDrive/option_price_prediction/data_preprocessing/data_08_18/input_08_18_exp1.npy')
y = np.load('/content/drive/MyDrive/option_price_prediction/data_preprocessing/data_08_18/output.npy')
print(X.shape, y.shape)

(100000, 20) (100000,)


In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [8]:
def divide_data(data, train_rate, val_rate, test_rate):
    train = data[0 : int(len(data) * train_rate)]
    val = data[int(len(data) * train_rate) : int(len(data) * (train_rate + val_rate))]
    test = data[int(len(data) * (train_rate + val_rate)) : len(data)]
    return train, val, test

In [9]:
train_X, val_X, test_X = divide_data(X, 0.6, 0.2, 0.2)
train_y, val_y, test_y = divide_data(y, 0.6, 0.2, 0.2)
print(train_X.shape)
print(train_y.shape)

(60000, 20)
(60000,)


In [20]:
# Build the model
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(train_X.shape[1], 1)))
model.add(LSTM(units=100, return_sequences=True))
model.add(LSTM(units=100, return_sequences=True))
model.add(LSTM(units=100, return_sequences=True))
model.add(Flatten())
model.add(Dense(units=1))

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
model.compile(optimizer='adam', loss='MeanSquaredError', metrics=\
              ['mean_absolute_error'])
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/option_price_prediction/model/08_18/exp1/model_checkpoint_exp2_{epoch:02d}.h5', save_freq='epoch')

In [21]:
history = model.fit(train_X, train_y, epochs=100, batch_size=64, validation_data=(val_X, val_y), callbacks=[checkpoint, reduce_lr])

Epoch 1/100
938/938 [==============================] - 20s 14ms/step - loss: 242.1908 - mean_absolute_error: 4.0527 - val_loss: 19.9538 - val_mean_absolute_error: 3.8622 - lr: 0.0010
Epoch 2/100
938/938 [==============================] - 14s 14ms/step - loss: 38.9541 - mean_absolute_error: 2.5998 - val_loss: 6.2002 - val_mean_absolute_error: 1.2464 - lr: 0.0010
Epoch 3/100
938/938 [==============================] - 14s 14ms/step - loss: 28.9651 - mean_absolute_error: 2.3037 - val_loss: 7.9424 - val_mean_absolute_error: 2.1666 - lr: 0.0010
Epoch 4/100
938/938 [==============================] - 14s 15ms/step - loss: 28.8050 - mean_absolute_error: 2.2756 - val_loss: 14.9006 - val_mean_absolute_error: 3.2699 - lr: 0.0010
Epoch 5/100
938/938 [==============================] - 14s 15ms/step - loss: 24.1012 - mean_absolute_error: 2.2331 - val_loss: 4.8312 - val_mean_absolute_error: 1.1854 - lr: 0.0010
Epoch 6/100
938/938 [==============================] - 12s 13ms/step - loss: 17.0977 - mean_

In [ ]:
model.save('/content/drive/MyDrive/option_price_prediction/model_08_18.h5')